In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rc
from pathlib import Path

from lowAltitude_classification.gsd_utils import papermode

In [ ]:
font_size = 8
papermode(plt, font_size, has_latex=True)
rc("text.latex", preamble=r"\usepackage{color}")
matplotlib.use("ps")

In [ ]:
# Load the data
cls_path = Path("lowAltitude_classification")
seg_path = Path("lowAltitude_segmentation")
data = pd.read_csv(cls_path / "BAR_CHART/bar_chart.csv")
data

In [ ]:
results_f1 = {
    r"Supervised" "\n" r"(\texttt{SP})": {
        "f1": data.iloc[0]["F1"],
        "use_drones": True,
        "use_pl": False,
        "img": "annotations/1.png",
    },
    r"Pseudo-labels" "\n" "(without voting)": {
        "f1": data.iloc[1]["F1"],
        "use_drones": False,
        "use_pl": False,
    },
    r"Pseudo-labels" "\n" "(with voting)": {
        "f1": data.iloc[2]["F1"],
        "use_drones": False,
        "use_pl": False,
        "img": "voting/1.png",
    },
    r"Pre-trained" "\n" r"(\texttt{PT})": {
        "f1": data.iloc[3]["F1"],
        "use_drones": False,
        "use_pl": True,
        "img": "pt/1.png",
    },
    r"Fine-tuned" "\n" r"(\texttt{FT})": {
        "f1": data.iloc[4]["F1"],
        "use_drones": True,
        "use_pl": True,
        "img": "pt_ft/1.png",
    },
}

# results_f1 = {k: round(v, 4) * 100 for k, v in results_f1.items()}
for k, v in results_f1.items():
    results_f1[k]["f1"] = round(v["f1"], 4) * 100

# width = 3.281
width = 4.3
height = width / 1.618
height = 1.75
fig, ax = plt.subplots(figsize=(width, height))
for i, (k, v) in enumerate(results_f1.items()):
    f1 = v["f1"]
    color = "teal"
    # if i == 0:
    #     color = 'maroon'
    if i == 1 or i == 2:
        color = "chocolate"
    bar = ax.bar(k, f1, color=color, edgecolor="black", zorder=1)
    # move the bar label down
    # bold text
    ax.bar_label(bar, color="white", zorder=2, padding=-12, fmt="%.2f")
    # , backgroundcolor=[1, 1, 1, 0.5], padding=3,
    # bbox=dict(facecolor=[1, 1, 1, 0.75], edgecolor='none', boxstyle='round,pad=0.2'))

    fig.text(
        bar[0].get_x() + bar[0].get_width() / 2,
        5,
        f"Drone",
        ha="center",
        va="bottom",
        color="white",
        fontsize=15,
    )

    if v["use_drones"]:
        ax.text(
            bar[0].get_x() + bar[0].get_width() / 2,
            3,
            r"\textbf{$D_{train}^{drone}$}%",
            ha="center",
            va="bottom",
            color="w",
            fontsize=font_size,
        )
    if v["use_pl"]:
        ax.text(
            bar[0].get_x() + bar[0].get_width() / 2,
            13,
            r"\textbf{$D^{pl}$}%",
            ha="center",
            va="bottom",
            color="w",
            fontsize=font_size,
        )

ax.set_yticks(range(0, 45, 10))
ax.grid(True, axis="y", color="gray", zorder=0, linestyle="--")
# ax.set_ylim(0, 50)
ax.set_axisbelow(True)
ax.set_ylabel(r"$F1$ Score on \hspace{2.2em} (\%)")
ax.text(
    -0.085,
    0.69,
    r"$D_{test}^{drone}$",
    fontsize=font_size,
    color="blue",
    transform=ax.transAxes,
    ha="center",
    va="center",
    rotation=90,
)

fig.subplots_adjust(top=0.99, bottom=0.23, left=0.095, right=0.99, wspace=0.3)
fig.set_size_inches(width, 0.8 * height)

fig.savefig(seg_path / "results/M2F_F1.pdf")
fig.savefig(seg_path / "results/M2F_F1.png")